In [1]:
import os
import re
import numpy as np
from sklearn.metrics import confusion_matrix

import torch
from torch.nn import MaxPool1d

In [3]:
dir_name = "causal_eval_kinetics_r50_i3d_baseline8_stream_centerCrop"
files_dir = "/Volumes/UBUNTU 18_0/PAKOVA/outputs/Kinetics/" + dir_name

result_files = sorted([os.path.join(files_dir, f) for f in os.listdir(files_dir) if f.endswith('.txt')])

split_text = []
for fid, fname in enumerate(result_files):
    print(fname)
    with open(fname, 'r', encoding ="ISO-8859-1") as infile:
#         a = infile.readline()
#         print(a)
        for line in infile:
            split_text.append(line.strip().split('|'))

/Volumes/UBUNTU 18_0/PAKOVA/outputs/Kinetics/causal_eval_kinetics_r50_i3d_baseline8_stream_centerCrop/causal_eval_kinetics_r50_i3d_baseline8_stream_centerCrop_0.txt
/Volumes/UBUNTU 18_0/PAKOVA/outputs/Kinetics/causal_eval_kinetics_r50_i3d_baseline8_stream_centerCrop/causal_eval_kinetics_r50_i3d_baseline8_stream_centerCrop_1.txt
/Volumes/UBUNTU 18_0/PAKOVA/outputs/Kinetics/causal_eval_kinetics_r50_i3d_baseline8_stream_centerCrop/causal_eval_kinetics_r50_i3d_baseline8_stream_centerCrop_2.txt
/Volumes/UBUNTU 18_0/PAKOVA/outputs/Kinetics/causal_eval_kinetics_r50_i3d_baseline8_stream_centerCrop/causal_eval_kinetics_r50_i3d_baseline8_stream_centerCrop_3.txt
/Volumes/UBUNTU 18_0/PAKOVA/outputs/Kinetics/causal_eval_kinetics_r50_i3d_baseline8_stream_centerCrop/causal_eval_kinetics_r50_i3d_baseline8_stream_centerCrop_4.txt
/Volumes/UBUNTU 18_0/PAKOVA/outputs/Kinetics/causal_eval_kinetics_r50_i3d_baseline8_stream_centerCrop/causal_eval_kinetics_r50_i3d_baseline8_stream_centerCrop_5.txt
/Volumes/U

In [4]:
split_text[0]

['driving_tractor/cn2OQeIdLcw.mp4_000089 ',
 ' 104 ',
 ' 104 103 201 372 199 ',
 ' 0.912668   0.01153553 0.0073093  0.00532365 0.00346765']

In [5]:
split_text2 = []
for i, line in enumerate(split_text):
    if len(line) == 1:
        aux = split_text2[-1][-1].replace(line[0], '') + ' ' + line[0]
        preds = np.fromstring(aux, dtype=float, sep=' ')
        split_text2[-1][-1] = aux
        
    elif len(line) == 4:
        split_text2.append(line)
    else:
        print(line)
        
%xdel split_text
split_text = split_text2

In [6]:
def get_full_predictions(ids, preds, num_classes=400):
    full_preds = np.zeros((num_classes))
    full_preds[ids] = preds
    return full_preds


def divide_per_clip(split_text):
    clips_targets = []
    clips_prediction = []

    video_name = None
    for i, [video_path, target, label, pred] in enumerate(split_text):
        video = re.sub('()_\\d{6}', '', video_path.strip())
        if video != video_name or i == len(split_text):            
            # new video! But first, save old video
            if i > 0:
                video_targets = np.array(video_targets)
                video_prediction = np.array(video_prediction)
                
                num_frames = video_prediction.shape[0]
                if num_frames < 10:
                    repeat_ids = np.round(np.linspace(0, num_frames - 1, 10)).astype(int)
                    video_targets = video_targets[repeat_ids]
                    video_prediction = video_prediction[repeat_ids]
                    
                clips_targets.append(video_targets)
                clips_prediction.append(video_prediction)
                    
            # star new one
            video_targets = []
            video_prediction = []
            video_name = video
            
        video_targets.append(int(target))
        labels = np.fromstring(label, dtype=int, sep=' ')
        preds = np.fromstring(pred, dtype=float, sep=' ')
        video_prediction.append(get_full_predictions(labels, preds))
                
    return clips_targets, clips_prediction

def select_n_clips(video_classes, n=10):
    num_frames = len(video_classes)
    ids = np.linspace(0, num_frames-1, n, dtype=int)
    
    return video_classes[ids]


In [7]:
# Dividing per clip
clip_targets, clip_pred = divide_per_clip(split_text)


# Getting result of 10 clips
clip_pred_10 = [select_n_clips(np.array(clip_data), n=10) for clip_data in clip_pred]

clip_pred_10_mean = np.array([t_c.mean(axis=0) for t_c in clip_pred_10])
clip_gt_10 = np.array([clip_data[0] for clip_data in clip_targets])


# Dividing all frames in percentages
p_clip_targets = np.array([np.array_split(ct, 10) for ct in clip_targets])
p_clip_pred = np.array([np.array_split(cp, 10) for cp in clip_pred])


In [8]:
num_videos = p_clip_targets.shape[0]
num_percent = p_clip_targets.shape[1]
num_topk = 5

percentage_targets = np.zeros((num_videos, num_percent))
percentage_pred = np.zeros((num_videos, num_percent, num_topk))

for clip_id, (p_targets, p_preds) in enumerate(zip(p_clip_targets, p_clip_pred)):
    for p_id in range(10):
        pt = np.hstack(p_targets[:p_id+1])
        pp = np.vstack(p_preds[:p_id+1])
        assert len(set(pt)) == 1  # Checking if all targets match
        percentage_targets[clip_id, p_id] = pt[0]
        preds = torch.tensor(pp)
        mean_pp = preds.mean(0)
        _, topk_labels = torch.topk(mean_pp, 5)
        percentage_pred[clip_id, p_id] = topk_labels.numpy()


In [9]:
def get_top_predictions(video_pred, video_labels):
    # TOP1
    top1_pred = [p[0] for p in video_pred]
    # TOP 5
    top5_pred = []
    for l, p in zip(video_labels, video_pred):
        if l in p:
            top5_pred.append(l)
        else:
            top5_pred.append(p[0])
    return top1_pred, top5_pred


def per_class_accuracy(predictions, labels):
    cf = confusion_matrix(labels, predictions).astype(float)

    cls_cnt = cf.sum(axis=1)
    cls_hit = np.diag(cf)
    return np.nanmean(cls_hit / cls_cnt)


def get_per_frame_percent_results(p_targets, p_preds):
    per_frame_percentage_results = []
    for p_id in range(10):
        p_t = p_targets[:, p_id]
        p_p = p_preds[:, p_id]
        top1_pred, top5_pred = get_top_predictions(p_p, p_t)

        cls_acc1 = per_class_accuracy(top1_pred, p_t)
        cls_acc5 = per_class_accuracy(top5_pred, p_t)
        per_frame_percentage_results.append([cls_acc1, cls_acc5])
        print((p_id+1)*10, cls_acc1, cls_acc5)
    return np.array(per_frame_percentage_results)

In [10]:
# 10 clips results

clip_top5_10 = []
for mean_10 in clip_pred_10_mean:
    _, topk_labels = torch.topk(torch.tensor(mean_10), 5)
    clip_top5_10.append(topk_labels.numpy())
    
clip_top5_10 = np.array(clip_top5_10)

top1_pred, top5_pred = get_top_predictions(clip_top5_10, clip_gt_10)

cls_acc1_10 = per_class_accuracy(top1_pred, clip_gt_10)
cls_acc5_10 = per_class_accuracy(top5_pred, clip_gt_10)

cls_acc1_10, cls_acc5_10

(0.7020829988562016, 0.891171358595158)

In [11]:
# All frames mean percentage results

per_frame_percent_res = get_per_frame_percent_results(percentage_targets, percentage_pred)

10 0.6055639672957738 0.8109984888973532
20 0.6222338067242006 0.8274481119413538
30 0.6431794008483752 0.8454186539815096
40 0.6620508723076872 0.8594927682661659
50 0.6756582378453581 0.8700037823145825
60 0.6833338483141554 0.8782477068401436
70 0.6910413349522523 0.8831455881230497
80 0.6961368434856072 0.8868778864599269
90 0.6993808926290359 0.8889059503982683
100 0.7010095130310975 0.8909292905966945


In [13]:
results = np.concatenate((np.array([[cls_acc1_10, cls_acc5_10]]), per_frame_percent_res))

In [14]:
output_file = files_dir + '_results'
np.save(output_file, results, allow_pickle=False)

In [15]:
results_baseline8_centerCrop = np.load(output_file+'.npy')

In [14]:
results_baseline32_centerCrop

array([[0.71854441, 0.89811049],
       [0.63530718, 0.84214335],
       [0.64962369, 0.85534666],
       [0.6685064 , 0.86938396],
       [0.68446872, 0.87948983],
       [0.6950135 , 0.88698023],
       [0.70064095, 0.89364684],
       [0.70767234, 0.89679222],
       [0.71330734, 0.89830761],
       [0.71603437, 0.89988545],
       [0.71950334, 0.90005517]])

In [14]:
results_nl32_centerCrop

array([[0.73355357, 0.91071791],
       [0.65183775, 0.85287248],
       [0.66693359, 0.8672611 ],
       [0.6822416 , 0.87995986],
       [0.69564461, 0.89168807],
       [0.7076758 , 0.89937124],
       [0.71635096, 0.90316556],
       [0.72185032, 0.90700233],
       [0.72674467, 0.9100338 ],
       [0.73233295, 0.91069292],
       [0.73417942, 0.91145335]])

In [16]:
results_baseline8_centerCrop

array([[0.702083  , 0.89117136],
       [0.60556397, 0.81099849],
       [0.62223381, 0.82744811],
       [0.6431794 , 0.84541865],
       [0.66205087, 0.85949277],
       [0.67565824, 0.87000378],
       [0.68333385, 0.87824771],
       [0.69104133, 0.88314559],
       [0.69613684, 0.88687789],
       [0.69938089, 0.88890595],
       [0.70100951, 0.89092929]])

In [89]:
results_nl8_centerCrop

array([[0.73935215, 0.91784009],
       [0.65289267, 0.85684554],
       [0.66969472, 0.86911939],
       [0.68593682, 0.88189664],
       [0.70082018, 0.89361338],
       [0.711213  , 0.90102226],
       [0.71994408, 0.90694043],
       [0.7256107 , 0.90999549],
       [0.73025136, 0.91263238],
       [0.73550584, 0.91540209],
       [0.73831541, 0.91741199]])